# ImgLib2 

Example 05: Out of bounds

## Introduction

Many algorithms, like convolutions, require to access pixels outside of an [**`Interval`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/Interval.html), i.e. also pixels outside of an image. In ImgLib2 this is handled using [**`Views`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html), which convert a [**`RandomAccessibleInterval`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessibleInterval.html) into an infinite [**`RandomAccessible`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html) using an [**`OutOfBoundsStrategy`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html). Those infinite [**`RandomAccessibles`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html) are able to return pixel values at any arbitrary location.

*Important: One should never access pixels outside of the defined [**`Interval`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/Interval.html) as it will in most cases result in unexpected behavior, depending on the kind of underlying [**`RandomAccessible`**]((http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html). If it is for example an [**`Img`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html), it will return wrong values or throw an exception.*

Which [**`OutOfBoundsStrategies`**](http://javadoc.imagej.net/ImgLib2/net/imglib2/view/Views.html) to use depends on the task you want to perform. For convolutions, we suggest using the mirror strategy as it introduces the least artifacts. When working on Fourier images, the periodic strategy applies best as it correctly mimics its spatial properties. Random Value strategies might be useful to avoid accidental correlations, and constant value strategies are the most performant and might work well for simple operations or to avoid exceptions when accidental writing or reading outside of the Interval occurs.

In [2]:
import io.scif.img.ImgOpener
import net.imglib2.RandomAccessible
import net.imglib2.img.Img
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.view.Views
import net.imglib2.FinalInterval
import net.imglib2.outofbounds.OutOfBoundsConstantValueFactory
import net.imglib2.view.ExtendedRandomAccessibleInterval

enum ExtendType {
  EXTEND_VALUE_0,
  EXTEND_VALUE_128,
  EXTEND_RANDOM_256,
  EXTEND_MIRROR_SINGLE,
  EXTEND_MIRROR_DOUBLE,
  EXTEND_PERIODIC,
  EXTEND_RANDOM_CUSTOM
}

// open with ImgOpener as a FloatType
image = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWingSmall.tif", new FloatType()).get(0)

// in order to visualize them, we have to define a new interval
// on them which can be displayed
min = new long[image.numDimensions()]
max = new long[image.numDimensions()]
 
for ( int d = 0; d < image.numDimensions(); ++d ) {
  // we add/subtract another 30 pixels here to illustrate
  // that it is really infinite and does not only work once
  min[d] = -image.dimension( d ) - 90
  max[d] = image.dimension( d ) * 2 - 1 + 90
}

// define the Interval on the infinite random accessibles
interval = new FinalInterval(min, max)

// change this variable to switch for different kind of out-of-bounds display
imgs = [:]
for (type in [ExtendType.EXTEND_VALUE_0,
    ExtendType.EXTEND_VALUE_128,
    ExtendType.EXTEND_RANDOM_256,
    ExtendType.EXTEND_MIRROR_SINGLE,
    ExtendType.EXTEND_MIRROR_DOUBLE,
    ExtendType.EXTEND_PERIODIC,
    ExtendType.EXTEND_RANDOM_CUSTOM])
{
    switch (type) {
      case ExtendType.EXTEND_VALUE_0:
        // create an infinite view where all values outside of the Interval are 0
        infinite = Views.extendValue(image, new FloatType(0))
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_VALUE_128:
        // create an infinite view where all values outside of the Interval are 128
        infinite = Views.extendValue(image, new FloatType(128))
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_RANDOM_256:
        // create an infinite view where all outside values are random in a range of 0-255
        infinite = Views.extendRandom(image, 0, 255);
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_MIRROR_SINGLE:
        // create an infinite view where all values outside of the Interval are
        // the mirrored content, the mirror is the last pixel
        infinite = Views.extendMirrorSingle(image);
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_MIRROR_DOUBLE:
        // create an infinite view where all values outside of the Interval are
        // the mirrored content, the mirror is BEHIND the last pixel,
        // i.e. the first and last pixel are always duplicated
        infinite = Views.extendMirrorDouble(image);
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_PERIODIC:
        // all values outside of the Interval periodically repeat the image content
        // (like the Fourier space assumes)
        infinite = Views.extendPeriodic(image);
        s = Views.interval(infinite, interval)
        break
      case ExtendType.EXTEND_RANDOM_CUSTOM:
        // if you implemented your own strategy that you want to instantiate, it will look like this
        infinite = new ExtendedRandomAccessibleInterval<FloatType, Img<FloatType>>(
          image, new OutOfBoundsConstantValueFactory<FloatType, Img<FloatType>>(new FloatType(196)));
        s = Views.interval(infinite, interval)
        break
      default:
        infinite = Views.extendValue(image, new FloatType(0))
        s = Views.interval(infinite, interval)
    }
    imgs.put(type, s)
}
imgs

Key,Value
EXTEND​_VALUE​_0,
EXTEND​_VALUE​_128,
EXTEND​_RANDOM​_256,
EXTEND​_MIRROR​_SINGLE,
EXTEND​_MIRROR​_DOUBLE,
EXTEND​_PERIODIC,
EXTEND​_RANDOM​_CUSTOM,
